## Code 4-1 無限深位能井

In [ ]:
%%capture
!pip install ipympl

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation
from scipy import linalg

%matplotlib widget

### 無限深方形阱

#### 畫出牆壁的位置
使用兩個單位階躍函數相減，就可以畫出井壁的位置
```
   ˍˍˍˍ|‾‾
-) ˍˍ|‾‾‾‾
=) ‾‾|ˍ|‾‾
```
注意此時頂端是0底部是-1，所以要再加上1

In [ ]:
def inf_barrier(x, width_nm=10, height=1):
    """無限深方形阱壁"""
    # width_nm: 阱壁寬度(nm)
    L = 1e-9 * width_nm  # 總距離(m)
    return (np.heaviside(x - L / 2, 0.5) - np.heaviside(x + L / 2, 0.5) + 1.) * height

In [ ]:
plt.clf()
x = np.linspace(-1e-8, 1e-8, 1000)

a = inf_barrier(10)
plt.plot(x, inf_barrier(x, 10))
plt.show()

### moving wave hit potential well

In [ ]:
def norm(psi, mesh_size):
    return np.sqrt(np.dot(np.conj(psi), psi) * mesh_size)

In [ ]:
def rho(psi):
    return (np.conj(psi) * psi).real

In [ ]:
def laplacian(n):
    """∇·∇運算子"""
    dx = x[1] - x[0]  # 微小變化量
    return (-2 * np.diag(np.ones(n, np.float32), 0)
            + np.diag(np.ones((n - 1), np.float32), 1)
            + np.diag(np.ones((n - 1), np.float32), -1)) / (dx ** 2)

In [ ]:
def barrier(avg_e=2.06, shape="square"):
    L = x[-1] - x[0]  # 總距離
    if shape == 'square':
        # __|▔|__
        pot = (np.heaviside(x - 0.45 * L, 0.5) - np.heaviside(x - 0.55 * L, 0.5)) * avg_e
    elif shape == 'heaviside':
        # ___|▔▔▔
        pot = np.heaviside(x - 0.5 * L, 0.5) * avg_e
    elif shape == 'well':
        # ▔▔|_|▔▔
        pot = (np.heaviside(x - 0.55 * L, 0.5) - np.heaviside(x - 0.45 * L, 0.5)) * avg_e
    else:
        pot = np.zeros_like(x)
    return pot

In [ ]:
def wave_packet(ax, kmu=2, ka=20):
    """kmu: peak momentum
       ka: momentum width parameter
       return the Fourier transformation of
              exp(-ka * (k - kmu)^2) * exp(-6j k^2)
    """
    L = ax[-1] - ax[0]  # x的最後項 - x的最前項
    dk = 2 * np.pi / L
    N = len(ax)
    k = np.linspace(0, N * dk, N)

    psi_k = np.exp(-ka * (k - kmu) ** 2 + -6j * k ** 2)  # Ψₖ

    temp = np.dot(np.diag(k * k, 0) / (2 * mass), psi_k)
    avg_e = np.dot(np.conjugate(psi_k), temp) * dk
    avg_e = avg_e / norm(psi_k, dk) ** 2

    psi = np.fft.ifft(psi_k)
    dx = x[1] - x[0]
    psi = psi / norm(psi, dx)
    return psi, avg_e

In [ ]:
def evolve(tfinal=30.0, nt=400):
    t = np.linspace(0, tfinal, nt)
    dt = t[1] - t[0]
    Ut = linalg.expm(-1j * H * dt / hbar)
    #print(f'{Ut=}')
    psi_list = []
    rho_list = []

    psi = np.copy(wave)
    psi_list.append(psi)
    rho_list.append(rho(psi))

    for i in range(nt):
        psi = np.dot(Ut, psi)
        psi_list.append(psi)
        rho_list.append(rho(psi))

    return t, psi_list, rho_list

In [ ]:
mass = 1
hbar = 1
xmin = 0
xmax = 100
ninterval = 1600
show_density = True

x = np.linspace(xmin, xmax, ninterval)
peak_shape = "none"

Lap = laplacian(ninterval)

In [ ]:
U = np.diag(barrier(shape=peak_shape), 0)
H = - hbar ** 2 / (2 * mass) * Lap + U

In [ ]:
wave, avgE = wave_packet(x, kmu=2, ka=20)

In [ ]:
plt.clf()
plt.grid(ls="--")
plt.plot(x, wave.real, label=r'$\psi(x)$')
if show_density:
    density = (np.conjugate(wave) * wave).real
    plt.plot(x, density, label='$\psi^*(x)\psi(x)$')
plt.xlabel(r'$x$')
plt.legend(loc='best', title="wave packet")
plt.show()

In [ ]:
t, psiList, rhoList = evolve(tfinal=30.0, nt=400)

In [ ]:
def update(i):
    line.set_data(x, psiList[i])
    text.set_text(f"${t[i]:.2f}$")
    return line, text


potential = barrier(shape=peak_shape)

fig1, ax1 = plt.subplots()
plt.plot(x, potential * 0.08)
line, = plt.plot(x, psiList[0])
text = plt.text(0, 0.05, '')
plt.grid(ls="--")

plt.ylabel('probability density')

plt.xlabel(r'$x$')
anim1 = FuncAnimation(fig1, update, frames=400, interval=100, blit=True)
anim1.save('o3.gif', fps=30, dpi=300)
plt.show()